<a href="https://colab.research.google.com/github/yohangumiel/Aulas-PUC-PR/blob/main/Computa%C3%A7%C3%A3o%20Forense/Computacao_Forense_Aula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Credits** to Miller P, Bryce C. Learning Python for Forensics: Leverage the power of Python in forensic investigations. Packt Publishing Ltd; 2019 Jan 31.

**Code adapted from**: https://github.com/PacktPublishing/Learning-Python-for-Forensics-Second-Edition



**Setupapi.dev.log** - Commonly examined because it extracts the first conecction time of USB devices on systems

**Log provides the installation time of a USB device's first connection, providing also the timestamp, the vendor ID (VID), the product ID (PID) and the serial number of the device.

STEPS:
1. Open the log file and read all lines (C:\Windows\inf\setupapi.dev.log)
2. In each line, check for indicators of a USB device entry
3. Search for timestamp and device information



In [8]:
import pandas as pd
import argparse
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen

"""
MIT License
Copyright (c) 2018 Chapin Bryce, Preston Miller
Please share comments and questions at:
    https://github.com/PythonForensics/Learning-Python-for-Forensics
    or email pyforcookbook@gmail.com
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

def get_usb_file():
    """
    Retrieves USB.ids database from the web.
    """
    url = 'http://www.linux-usb.org/usb.ids'
    return urlopen(url)


def parse_file(usb_file):
    """
    Parses the USB.ids file. If this is run offline, please download the
    USB.ids and pass the open file to this function.
    ie: parse_file(open('path/to/USB.ids', 'r'))
    :return: dictionary of entires for querying
    """
    usbs = {}
    curr_id = ''
    for line in usb_file:
        if isinstance(line, bytes):
            line = line.decode('latin-1')
        if line.startswith('#') or line in ('\n', '\t'):
            continue
        else:
            if not line.startswith('\t') and (line[0].isdigit() or
                                              line[0].islower()):
                uid, name = get_record(line.strip())
                curr_id = uid
                usbs[uid] = [name.strip(), {}]
            elif line.startswith('\t') and line.count('\t') == 1:
                uid, name = get_record(line.strip())
                usbs[curr_id][1][uid] = name.strip()
    return usbs


def get_record(record_line):
    """
    Split records out by dynamic position. By finding the space, we can
    determine the location to split the record for extraction. To learn more
    about this, uncomment the print statements and see what the code is
    doing behind the scenes!
    """
    # print("Line: {}".format(record_line))
    split = record_line.find(' ')
    # print("Split: {}".format(split))
    record_id = record_line[:split]
    # print("Record ID: ".format(record_id))
    record_name = record_line[split + 1:]
    # print("Record Name: ".format(record_name))
    return record_id, record_name


def search_key(usb_dict, ids):
    """
    Compare provided IDs to the built USB dictionary. If found, it will
    return the common name, otherwise returns the string "unknown".
    """
    vendor_key = ids[0]
    product_key = ids[1]

    vendor, vendor_data = usb_dict.get(vendor_key, ['unknown', {}])
    product = 'unknown'
    if vendor != 'unknown':
        product = vendor_data.get(product_key, 'unknown')

    return vendor, product

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/setupapi.dev.log'

data = []
for line in open(file_path, errors= 'ignore').readlines():
    data.append(line)

data[50:100]


['     inf:                     Section Name = DefaultInstall.NTamd64\n',
 '     inf:                     Driver Type  = Primitive\n',
 '     sto:                     Finding all driver package versions\n',
 '     sto:                          gameflt.inf_amd64_522fdd92caccac75 [oem100.inf] - 11/03/2021,10.0.22002.6 (Active)\n',
 "     sto:                     Selected driver package 'gameflt.inf_amd64_522fdd92caccac75'.\n",
 '     inf:                     {Add Service: gameflt}\n',
 '     inf:                          Start Type    = 3\n',
 '     inf:                          Service Type  = 2\n',
 '     inf:                          Error Control = 1\n',
 '     inf:                          Image Path    = \\SystemRoot\\System32\\DriverStore\\FileRepository\\gameflt.inf_amd64_522fdd92caccac75\\gameflt.sys\n',
 '     inf:                          Display Name  = gameflt\n',
 '     inf:                          Description   = Game mini-filter driver\n',
 '     inf:                    

#### Device and timestamp

In [4]:
for i, line in enumerate(data): # enumerate to get the next element in list

    if 'Device Install (Hardware initiated)' in line: #possibility for regex
        print('*****DATA****\n')
        print(line)
        device_name = data[i].split('-')[1].strip()
        print('DEVICE NAME:', device_name[:-1],'\n')
        print('TIME DATA', data[i+1])
        date = data[i+1].split('start')[1].strip()
        print('TIMESTAMP: ',date,'\n')
        print('\n********\n')

*****DATA****

>>>  [Device Install (Hardware initiated) - USB\VID_046D&PID_0A5B&MI_00\6&806f8d6&0&0000]

DEVICE NAME: USB\VID_046D&PID_0A5B&MI_00\6&806f8d6&0&0000 

TIME DATA >>>  Section start 2022/01/09 14:39:07.527

TIMESTAMP:  2022/01/09 14:39:07.527 


********

*****DATA****

>>>  [Device Install (Hardware initiated) - USB\VID_046D&PID_0A5B&MI_00\6&30e81238&1&0000]

DEVICE NAME: USB\VID_046D&PID_0A5B&MI_00\6&30e81238&1&0000 

TIME DATA >>>  Section start 2022/01/14 14:23:35.189

TIMESTAMP:  2022/01/14 14:23:35.189 


********

*****DATA****

>>>  [Device Install (Hardware initiated) - USB\VID_046D&PID_0A66&MI_00\6&dffd75b&0&0000]

DEVICE NAME: USB\VID_046D&PID_0A66&MI_00\6&dffd75b&0&0000 

TIME DATA >>>  Section start 2022/01/23 10:17:13.868

TIMESTAMP:  2022/01/23 10:17:13.868 


********

*****DATA****

>>>  [Device Install (Hardware initiated) - USB\VID_1532&PID_0068&MI_02\6&145a1bd4&0&0002]

DEVICE NAME: USB\VID_1532&PID_0068&MI_02\6&145a1bd4&0&0002 

TIME DATA >>>  Section 

Many devices including PCI and not only USB devices

In [12]:
  for i, line in enumerate(data): # enumerate to get the next element in list

    if 'Device Install (Hardware initiated)' in line and ('VEN' in line or 'VID' in line):
        
        device_name = data[i].split('-')[1].strip()


        device = device_name.split('\\')[0].lower()
        print('DEVICE: ', device)
        if 'usb' in device:  
            print('DEVICE NAME:', device_name[:-1],'\n')
            print('TIME DATA', data[i+1])
            date = data[i+1].split('start')[1].strip()
            print('TIMESTAMP: ',date,'\n')
            print('\n********\n')

    


DEVICE:  usb
DEVICE NAME: USB\VID_046D&PID_0A5B&MI_00\6&806f8d6&0&0000 

TIME DATA >>>  Section start 2022/01/09 14:39:07.527

TIMESTAMP:  2022/01/09 14:39:07.527 


********

DEVICE:  usb
DEVICE NAME: USB\VID_046D&PID_0A5B&MI_00\6&30e81238&1&0000 

TIME DATA >>>  Section start 2022/01/14 14:23:35.189

TIMESTAMP:  2022/01/14 14:23:35.189 


********

DEVICE:  usb
DEVICE NAME: USB\VID_046D&PID_0A66&MI_00\6&dffd75b&0&0000 

TIME DATA >>>  Section start 2022/01/23 10:17:13.868

TIMESTAMP:  2022/01/23 10:17:13.868 


********

DEVICE:  usb
DEVICE NAME: USB\VID_1532&PID_0068&MI_02\6&145a1bd4&0&0002 

TIME DATA >>>  Section start 2022/01/23 10:21:24.246

TIMESTAMP:  2022/01/23 10:21:24.246 


********

DEVICE:  usb
DEVICE NAME: USB\VID_0951&PID_1642\001D6004CAC2BC2155AE0093 

TIME DATA >>>  Section start 2022/01/26 13:40:52.202

TIMESTAMP:  2022/01/26 13:40:52.202 


********

DEVICE:  usb
DEVICE NAME: USB\VID_1532&PID_0068&MI_02\6&1a06b2f&0&0002 

TIME DATA >>>  Section start 2022/02/24 08:

#### Search for VEN and VID information (Vendor ID)

In [25]:
 device_list = []
 unique_devices = []
 for i, line in enumerate(data): # enumerate to get the next element in list

    if 'Device Install (Hardware initiated)' in line and ('VEN' in line or 'VID' in line):
        device_name = data[i].split('-')[1].strip()
        device = device_name.split('\\')[0].lower()
        if 'usb' in device:  
            date = data[i+1].split('start')[1].strip()
            device_list.append([device_name, date])

df = pd.DataFrame(device_list, columns = ['device','date'])
df

,device,date
0,USB\VID_046D&PID_0A5B&MI_00\6&806f8d6&0&0000],2022/01/09 14:39:07.527
1,USB\VID_046D&PID_0A5B&MI_00\6&30e81238&1&0000],2022/01/14 14:23:35.189
2,USB\VID_046D&PID_0A66&MI_00\6&dffd75b&0&0000],2022/01/23 10:17:13.868
3,USB\VID_1532&PID_0068&MI_02\6&145a1bd4&0&0002],2022/01/23 10:21:24.246
4,USB\VID_0951&PID_1642\001D6004CAC2BC2155AE0093],2022/01/26 13:40:52.202
5,USB\VID_1532&PID_0068&MI_02\6&1a06b2f&0&0002],2022/02/24 08:37:27.632
6,USB\VID_046D&PID_0A66&MI_00\6&dffd75b&0&0000],2022/03/28 20:13:08.643


In [31]:
device_list

[['USB\\VID_046D&PID_0A5B&MI_00\\6&806f8d6&0&0000]',
  '2022/01/09 14:39:07.527'],
 ['USB\\VID_046D&PID_0A5B&MI_00\\6&30e81238&1&0000]',
  '2022/01/14 14:23:35.189'],
 ['USB\\VID_046D&PID_0A66&MI_00\\6&dffd75b&0&0000]',
  '2022/01/23 10:17:13.868'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&145a1bd4&0&0002]',
  '2022/01/23 10:21:24.246'],
 ['USB\\VID_0951&PID_1642\\001D6004CAC2BC2155AE0093]',
  '2022/01/26 13:40:52.202'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&1a06b2f&0&0002]',
  '2022/02/24 08:37:27.632']]

In [32]:
 for device_element in device_list:
    # Initialize variables
    vid = ''
    pid = ''
    rev = ''
    uid = ''

    # Split string into segments on \\
    segments = device_element[0].split('\\')

    for item in segments[1].split('&'):
        print(item)
        lower_item = item.lower()
        if 'ven' in lower_item or 'vid' in lower_item:
            vid = item.split('_', 1)[-1]
        elif 'dev' in lower_item or 'pid' in lower_item or \
                'prod' in lower_item:
            pid = item.split('_', 1)[-1]
        elif 'rev' in lower_item or 'mi' in lower_item:
            rev = item.split('_', 1)[-1]

    if len(segments) >= 3:
        print(segments[2])
        uid = segments[2].strip(']')

    if vid != '' or pid != '':
        device_element.append(vid.lower())
        device_element.append(pid.lower())
        device_element.append(rev)
        device_element.append(uid)

VID_046D
PID_0A5B
MI_00
6&806f8d6&0&0000]
VID_046D
PID_0A5B
MI_00
6&30e81238&1&0000]
VID_046D
PID_0A66
MI_00
6&dffd75b&0&0000]
VID_1532
PID_0068
MI_02
6&145a1bd4&0&0002]
VID_0951
PID_1642
001D6004CAC2BC2155AE0093]
VID_1532
PID_0068
MI_02
6&1a06b2f&0&0002]


In [33]:
print('****DEVICES****')
device_list

****DEVICES****


[['USB\\VID_046D&PID_0A5B&MI_00\\6&806f8d6&0&0000]',
  '2022/01/09 14:39:07.527',
  '046d',
  '0a5b',
  '00',
  '6&806f8d6&0&0000'],
 ['USB\\VID_046D&PID_0A5B&MI_00\\6&30e81238&1&0000]',
  '2022/01/14 14:23:35.189',
  '046d',
  '0a5b',
  '00',
  '6&30e81238&1&0000'],
 ['USB\\VID_046D&PID_0A66&MI_00\\6&dffd75b&0&0000]',
  '2022/01/23 10:17:13.868',
  '046d',
  '0a66',
  '00',
  '6&dffd75b&0&0000'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&145a1bd4&0&0002]',
  '2022/01/23 10:21:24.246',
  '1532',
  '0068',
  '02',
  '6&145a1bd4&0&0002'],
 ['USB\\VID_0951&PID_1642\\001D6004CAC2BC2155AE0093]',
  '2022/01/26 13:40:52.202',
  '0951',
  '1642',
  '',
  '001D6004CAC2BC2155AE0093'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&1a06b2f&0&0002]',
  '2022/02/24 08:37:27.632',
  '1532',
  '0068',
  '02',
  '6&1a06b2f&0&0002']]

## Get usb_file and parse it

In [34]:
usb_file = get_usb_file()
usbs = parse_file(usb_file)

In [35]:
usbs

{'0001': ["Fry's Electronics", {'7778': 'Counterfeit flash drive [Kingston]'}],
 '0002': ['Ingram', {'0002': 'passport00'}],
 '0003': ['Club Mac', {}],
 '0004': ['Nebraska Furniture Mart', {}],
 '0011': ['Unknown', {'7788': 'counterfeit flash drive'}],
 '0053': ['Planex', {'5301': 'GW-US54ZGL 802.11bg'}],
 '0078': ['Microntek', {'0006': 'Joystick'}],
 '0079': ['DragonRise Inc.',
  {'0006': 'PC TWIN SHOCK Gamepad',
   '0011': 'Gamepad',
   '1800': 'Mayflash Wii U Pro Game Controller Adapter [DirectInput]',
   '181b': 'Venom Arcade Joystick',
   '1843': 'Mayflash GameCube Controller Adapter',
   '1844': 'Mayflash GameCube Controller'}],
 '0080': ['Unknown', {'a001': 'JMS578 based SATA bridge'}],
 '0085': ['Boeye Technology Co., Ltd.', {'0600': 'eBook Reader'}],
 '0102': ['miniSTREAK', {}],
 '0105': ['Trust International B.V.',
  {'145f': 'NW-3100 802.11b/g 54Mbps Wireless Network Adapter [zd1211]'}],
 '0127': ['IBP', {'0002': 'HDM Interface', '0127': 'ibp'}],
 '0145': ['Unknown', {'0112'

## Lookup for information based on VID/PID (Vendor ID and Product ID)

In [36]:
for device_element in device_list:

    vendor_name = ''
    product_name = ''

    device_name = search_key(usbs, [device_element[2],device_element[3]])

    device_element.append(device_name[0])
    device_element.append(device_name[1])

In [37]:
device_list

[['USB\\VID_046D&PID_0A5B&MI_00\\6&806f8d6&0&0000]',
  '2022/01/09 14:39:07.527',
  '046d',
  '0a5b',
  '00',
  '6&806f8d6&0&0000',
  'Logitech, Inc.',
  'G933 Wireless Headset Dongle'],
 ['USB\\VID_046D&PID_0A5B&MI_00\\6&30e81238&1&0000]',
  '2022/01/14 14:23:35.189',
  '046d',
  '0a5b',
  '00',
  '6&30e81238&1&0000',
  'Logitech, Inc.',
  'G933 Wireless Headset Dongle'],
 ['USB\\VID_046D&PID_0A66&MI_00\\6&dffd75b&0&0000]',
  '2022/01/23 10:17:13.868',
  '046d',
  '0a66',
  '00',
  '6&dffd75b&0&0000',
  'Logitech, Inc.',
  '[G533 Wireless Headset Dongle]'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&145a1bd4&0&0002]',
  '2022/01/23 10:21:24.246',
  '1532',
  '0068',
  '02',
  '6&145a1bd4&0&0002',
  'Razer USA, Ltd',
  'Gaming Mouse Mat [Firefly Hyperflux]'],
 ['USB\\VID_0951&PID_1642\\001D6004CAC2BC2155AE0093]',
  '2022/01/26 13:40:52.202',
  '0951',
  '1642',
  '',
  '001D6004CAC2BC2155AE0093',
  'Kingston Technology',
  'DT101 G2'],
 ['USB\\VID_1532&PID_0068&MI_02\\6&1a06b2f&0&0002]',
  '20

## Print USB device information

In [38]:
for element in device_list:
    print('Vendor ID: ', element[2])
    print('Vendor: ', element[6])
    print('Product ID: ', element[3])   
    print('Product: ', element[7])     
    print('Revision: ', element[4]) 
    print('UID: ', element[5])
    print('First Installation Date', element[1])
    print('\n********\n')

Vendor ID:  046d
Vendor:  Logitech, Inc.
Product ID:  0a5b
Product:  G933 Wireless Headset Dongle
Revision:  00
UID:  6&806f8d6&0&0000
First Installation Date 2022/01/09 14:39:07.527

********

Vendor ID:  046d
Vendor:  Logitech, Inc.
Product ID:  0a5b
Product:  G933 Wireless Headset Dongle
Revision:  00
UID:  6&30e81238&1&0000
First Installation Date 2022/01/14 14:23:35.189

********

Vendor ID:  046d
Vendor:  Logitech, Inc.
Product ID:  0a66
Product:  [G533 Wireless Headset Dongle]
Revision:  00
UID:  6&dffd75b&0&0000
First Installation Date 2022/01/23 10:17:13.868

********

Vendor ID:  1532
Vendor:  Razer USA, Ltd
Product ID:  0068
Product:  Gaming Mouse Mat [Firefly Hyperflux]
Revision:  02
UID:  6&145a1bd4&0&0002
First Installation Date 2022/01/23 10:21:24.246

********

Vendor ID:  0951
Vendor:  Kingston Technology
Product ID:  1642
Product:  DT101 G2
Revision:  
UID:  001D6004CAC2BC2155AE0093
First Installation Date 2022/01/26 13:40:52.202

********

Vendor ID:  1532
Vendor:  R